In [1]:
import pandas as pd
import ast
import json


In [6]:

def missing_values_table(df):
    '''
    Se pasa como parametro un df de pandas, devuelve la 
    cantidad de valores NaN y a que porcentaje del total de valores corresponen
    
    
    '''
    
    mis_val = df.isna().sum()
    mis_val_percent = 100 * df.isna().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns




# IMPORTANDO DATAFRAMES

In [ ]:
lineas = []   #Creamos una lista vacia 

with open('data/australian_user_reviews.json', encoding='utf-8') as f:  #iteramos sobre cada uno de las lineas del DF y rellenamos la lista vacia con cada iteracion
    for line in f.readlines():
        lineas.append(ast.literal_eval(line))
        
user_reviews = pd.DataFrame(lineas)  #creamos el DF con los datos de la lista

In [3]:
lineas = []

with open('data/australian_users_items.json', encoding='utf-8') as f:
    for line in f.readlines():
        lineas.append(ast.literal_eval(line))
        
user_items = pd.DataFrame(lineas)

In [3]:
#creamos el DF con pandas.
steam_games = pd.read_json('data\output_steam_games.json', lines = True)  

In [107]:
steam_games.head()

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [7]:
user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


##### EXPORTANDO DF EN CSV

Como sabia que iba a ser un porceso muy iterativo, lleno de mucha prueba y error, decidí exportar los tres dataframes a csv ya que considero muchisimo mas sencilla la carga de datos en este formato

In [9]:
user_reviews.to_csv('user_reviews.csv')

In [10]:
user_items.to_csv('user_items.csv')

In [23]:
steam_games.to_csv('steam_games.csv')

# TRATANDO LOS DATOS

In [ ]:
#cargando CSV's
df_reviews = pd.read_csv(r'data\user_reviews.csv')
df_items = pd.read_csv(r'data\user_items.csv')
df_games = pd.read_csv(r'data\steam_games.csv')

### user_reviews



In [94]:
df_reviews.head()

,Unnamed: 0,user_id,user_url,reviews
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


La columna reviews esta llena de valores anidados. Tendriamos que desanidar esa columna.

Tambien vemos que los campos de esta columna estan en formato srt, lo cual dificulta muchisimo su tratamiento

In [95]:
type(df_reviews['reviews'][0])

str

In [96]:
#aplicamos ast.literal_eval a toda la columna.
#OJO, puede llegar a usar mucha memoria y comer muchos recursos
df_reviews['parsed_data'] = df_reviews['reviews'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Utilizar explode en la columna 'parsed_data' para desanidar los diccionarios
df_exploded = df_reviews.explode('parsed_data', ignore_index=True)

Logramos que la columna este llena de diccionarios y no de listas de diccionarios

In [97]:
type(df_exploded['parsed_data'][0])

dict

In [98]:
df_exploded.head()

,Unnamed: 0,user_id,user_url,reviews,parsed_data
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","{'funny': '', 'posted': 'Posted November 5, 20..."
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","{'funny': '', 'posted': 'Posted July 15, 2011...."
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","{'funny': '', 'posted': 'Posted April 21, 2011..."
3,1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","{'funny': '', 'posted': 'Posted June 24, 2014...."
4,1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","{'funny': '', 'posted': 'Posted September 8, 2..."


In [99]:
#normalizamos, concatenamos las columnas nuevas y finalmente deshacemos las columnas viejas. NOTA: "Unnamed: 0" va a aparecer mucho
reviews_normalized = pd.json_normalize(df_exploded['parsed_data'])
df_reviews_final = pd.concat([df_exploded,reviews_normalized], axis=1)
df_reviews_final.drop(columns=['Unnamed: 0', 'reviews', 'parsed_data'], inplace=True)

In [100]:
#testeando
df_reviews_final.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [101]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


In [102]:
df_reviews_final.to_csv("df_reviews.csv")

### user_items

In [98]:
df_items.head()

,Unnamed: 0,user_id,items_count,steam_id,user_url,items
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


La columna items esta llena de valores anidados. Tendriamos que desanidar esa columna.  Tambien tenemos el mismo problema, los campos estan llenos de strings los cuales no nos permiten trabajar

In [99]:
#reutilizamos el codigo que escribimos arriba para parsear la columna
df_items['parsed_data'] = df_items['items'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Utilizar explode en la columna 'parsed_data' para desanidar los diccionarios
df_items = df_items.explode('parsed_data', ignore_index=True)

In [100]:
df_items.head()

,Unnamed: 0,user_id,items_count,steam_id,user_url,items,parsed_data
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","{'item_id': '10', 'item_name': 'Counter-Strike..."
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","{'item_id': '20', 'item_name': 'Team Fortress ..."
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [101]:
items_normalized = pd.json_normalize(df_items['parsed_data'])
df_items_final = pd.concat([df_items,items_normalized], axis=1)
df_items_final.drop(columns=['Unnamed: 0', 'items', 'parsed_data'], inplace=True)

In [102]:

df_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          int64  
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 315.6+ MB


In [105]:
#exportamos para utilizar mas adelante
df_items_final.to_csv('df_items.csv')

### steam_games

In [3]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    120445 non-null  int64  
 1   publisher     24064 non-null   object 
 2   genres        28852 non-null   object 
 3   app_name      32133 non-null   object 
 4   title         30085 non-null   object 
 5   url           32135 non-null   object 
 6   release_date  30068 non-null   object 
 7   tags          31972 non-null   object 
 8   reviews_url   32133 non-null   object 
 9   specs         31465 non-null   object 
 10  price         30758 non-null   object 
 11  early_access  32135 non-null   float64
 12  id            32133 non-null   float64
 13  developer     28836 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 12.9+ MB


In [4]:
df_games.head()

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


esta funcion esta definida en la celda 2. Sirve para contar cuantos valores nulos hay en cada columna y que porcentaje ocupa en este caso vemos que son un monton

In [7]:
missing_values_table(df_games)

Your selected dataframe has 14 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values
publisher,96381,80.0
developer,91609,76.1
genres,91593,76.0
release_date,90377,75.0
title,90360,75.0
price,89687,74.5
specs,88980,73.9
tags,88473,73.5
app_name,88312,73.3
reviews_url,88312,73.3


In [8]:
#conteo de : "Unnamed: 0" = 2
df_games.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
#eliminamos las primeras 88.000 columnas que estaban llenas de NaN
df_games.dropna(how='all', inplace=True)

In [10]:
#mejoró bastante
missing_values_table(df_games)

Your selected dataframe has 13 columns.
There are 11 columns that have missing values.


,Missing Values,% of Total Values
publisher,8071,25.1
developer,3299,10.3
genres,3283,10.2
release_date,2067,6.4
title,2050,6.4
price,1377,4.3
specs,670,2.1
tags,163,0.5
app_name,2,0.0
reviews_url,2,0.0


Ahora lo primero que veo es que tenemos dos columnas que son muy similares y refieren a lo mismo(al nombre del juego) por lo tanto elijo borrar app_name pero antes utilizandola para intentar rellenar algunos valores faltantes de la columna tittle

In [11]:

df_games.sample(10)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
107814,DarkDes Labs,"['Action', 'Adventure', 'Indie', 'RPG']",Vzerthos: The Heir of Thunder,Vzerthos: The Heir of Thunder,http://store.steampowered.com/app/536450/Vzert...,2017-01-19,"['Action', 'Adventure', 'Indie', 'Violent', 'R...",http://steamcommunity.com/app/536450/reviews/?...,"['Single-player', 'Steam Achievements', 'Full ...",6.99,0.0,536450.0,DarkDes Labs
88601,Sakari Indie,"['Action', 'Indie']",Foreign Legion: Buckets of Blood,Foreign Legion: Buckets of Blood,http://store.steampowered.com/app/36000/Foreig...,2009-08-04,"['Action', 'Indie', 'Third Person', 'Shooter']",http://steamcommunity.com/app/36000/reviews/?b...,"['Single-player', 'Steam Achievements', 'Parti...",4.99,0.0,36000.0,Sakari Indie
108023,NaN,"['Casual', 'Simulation']",Rocksmith® 2014 Edition – Remastered – U2 - “S...,Rocksmith® 2014 Edition – Remastered – U2 - “S...,http://store.steampowered.com/app/509690/Rocks...,2017-01-03,"['Casual', 'Simulation']",http://steamcommunity.com/app/509690/reviews/?...,"['Single-player', 'Shared/Split Screen', 'Down...",2.99,0.0,509690.0,Ubisoft - San Francisco
92049,Mudvark,"['Adventure', 'Indie']",Rituals,Rituals,http://store.steampowered.com/app/359190/Rituals/,2015-05-27,"['Adventure', 'Indie', 'Point & Click', 'Puzzle']",http://steamcommunity.com/app/359190/reviews/?...,"['Single-player', 'Steam Achievements']",4.99,0.0,359190.0,Tymon Zgainski
103143,NaN,"['Indie', 'Simulation']",A Mortician's Tale Original Soundtrack,A Mortician's Tale Original Soundtrack,http://store.steampowered.com/app/736320/A_Mor...,2017-10-18,"['Indie', 'Simulation']",http://steamcommunity.com/app/736320/reviews/?...,"['Single-player', 'Downloadable Content', 'Ste...",4.99,0.0,736320.0,Laundry Bear Games
93687,Workyrie Game Studio,['RPG'],Wanderjahr,Wanderjahr,http://store.steampowered.com/app/434860/Wande...,2016-02-18,"['RPG', 'Anime']",http://steamcommunity.com/app/434860/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",9.99,0.0,434860.0,Workyrie Game Studio
94017,Ankama Games,['Action'],ABRACA - Imagic Games,ABRACA - Imagic Games,http://store.steampowered.com/app/345990/ABRAC...,2016-03-31,"['Action', '4 Player Local', 'Local Multiplaye...",http://steamcommunity.com/app/345990/reviews/?...,"['Shared/Split Screen', 'Steam Achievements', ...",14.99,0.0,345990.0,Ankama Studio
114062,Lightning Man Media,"['Adventure', 'Indie']",Nyctophobia,Nyctophobia,http://store.steampowered.com/app/378100/Nycto...,2015-08-25,"['Indie', 'Adventure', 'Horror']",http://steamcommunity.com/app/378100/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,0.0,378100.0,Lightning Man Media
108452,HexWar Games,['Strategy'],Tank Battle: Blitzkrieg,Tank Battle: Blitzkrieg,http://store.steampowered.com/app/540150/Tank_...,2016-12-07,['Strategy'],http://steamcommunity.com/app/540150/reviews/?...,['Single-player'],9.99,0.0,540150.0,HexWar Games
95980,Farsky Interactive,"['Action', 'Adventure', 'Indie', 'RPG']",Sky Break,Sky Break,http://store.steampowered.com/app/405370/Sky_B...,2016-10-21,"['RPG', 'Indie', 'Adventure', 'Open World', 'S...",http://steamcommunity.com/app/405370/reviews/?...,"['Single-player', 'Partial Controller Support']",9.99,0.0,405370.0,Farsky Interactive


In [12]:
#rellenamos los valores faltantes de la columna "titulo" con los valores de la columna "app_name".
df_games['title'] = df_games['title'].fillna(df_games['app_name'])

In [13]:
#elimino la columna "app_name" ya la pude combinar con la columna title
df_games.drop(columns=['app_name'], inplace= True)

In [14]:
#buscamos valores faltantes
missing_values_table(df_games)

Your selected dataframe has 12 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
publisher,8071,25.1
developer,3299,10.3
genres,3283,10.2
release_date,2067,6.4
price,1377,4.3
specs,670,2.1
tags,163,0.5
title,2,0.0
reviews_url,2,0.0
id,2,0.0


In [15]:
#chequeamos los valores unicos para hacer replace() y poder cambiar el tipo de datos a uno manipulable.
print(pd.unique(df_games['price']))

['4.99' 'Free To Play' 'Free to Play' '0.99' '2.99' '3.99' '9.99' '18.99'
 '29.99' nan 'Free' '10.99' '1.5899999999999999' '14.99' '1.99' '59.99'
 '8.99' '6.99' '7.99' '39.99' '19.99' '7.49' '12.99' '5.99' '2.49' '15.99'
 '1.25' '24.99' '17.99' '61.99' '3.49' '11.99' '13.99' 'Free Demo'
 'Play for Free!' '34.99' '74.76' '1.49' '32.99' '99.99' '14.95' '69.99'
 '16.99' '79.99' '49.99' '5.0' '44.99' '13.98' '29.96' '119.99' '109.99'
 '149.99' '771.71' 'Install Now' '21.99' '89.99'
 'Play WARMACHINE: Tactics Demo' '0.98' '139.92' '4.29' '64.99' 'Free Mod'
 '54.99' '74.99' 'Install Theme' '0.89' 'Third-party' '0.5' 'Play Now'
 '299.99' '1.29' '3.0' '15.0' '5.49' '23.99' '49.0' '20.99' '10.93'
 '1.3900000000000001' 'Free HITMAN™ Holiday Pack' '36.99' '4.49' '2.0'
 '4.0' '9.0' '234.99' '1.9500000000000002' '1.5' '199.0' '189.0' '6.66'
 '27.99' '10.49' '129.99' '179.0' '26.99' '399.99' '31.99' '399.0' '20.0'
 '40.0' '3.33' '199.99' '22.99' '320.0' '38.85' '71.7' '59.95' '995.0'
 '27.49' '3.39'

In [16]:
#que significa "third-party"? ambos juegos son Free To Play, asi que podemos encapsularlos dentro de esa categoria
df_games[df_games['price'] == 'Third-party']

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
92228,NaN,['Indie'],Parcel - Soundtrack,http://store.steampowered.com/app/362970/Parce...,2015-07-02,['Indie'],http://steamcommunity.com/app/362970/reviews/?...,"['Single-player', 'Shared/Split Screen', 'Down...",Third-party,0.0,362970.0,Polar Bunny Ltd
120148,"PopCap Games, Inc.",['Casual'],Peggle Extreme,http://store.steampowered.com/app/3483/Peggle_...,2007-09-11,"['Casual', 'Puzzle', 'Free to Play', 'Action']",http://steamcommunity.com/app/3483/reviews/?br...,['Single-player'],Third-party,0.0,3483.0,"PopCap Games, Inc."


In [17]:
#decido que los juegos Free to Play y otros similares, en vez de tener esa leyenda sea de precio 0.00, asi puedo tratarlos de manera mas sencilla mas adelante.

df_games.replace(['Free To Play', 'Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now',
                  'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now',
                  'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use'], '0.00', inplace=True)

In [18]:
#consola, no video juego
df_games[df_games['price'] == 'Starting at $499.00']

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
113310,NaN,NaN,Syber Steam Machine,http://store.steampowered.com/app/353420/Syber...,2015-11-10,"['Steam Machine', 'Hardware']",http://steamcommunity.com/app/353420/reviews/?...,NaN,Starting at $499.00,0.0,353420.0,NaN


In [19]:
#tampoco es un juego, es otra consola
df_games[df_games['price'] == 'Starting at $449.00']

,publisher,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
113311,NaN,NaN,Alienware Steam Machine,http://store.steampowered.com/app/353390/Alien...,2015-11-10,"['Steam Machine', 'Hardware', 'Gaming', 'Futur...",http://steamcommunity.com/app/353390/reviews/?...,NaN,Starting at $449.00,0.0,353390.0,NaN


In [20]:
df_games.drop([113310, 113311], inplace=True)

In [21]:
print(pd.unique(df_games['price']))

['4.99' '0.00' '0.99' '2.99' '3.99' '9.99' '18.99' '29.99' nan '10.99'
 '1.5899999999999999' '14.99' '1.99' '59.99' '8.99' '6.99' '7.99' '39.99'
 '19.99' '7.49' '12.99' '5.99' '2.49' '15.99' '1.25' '24.99' '17.99'
 '61.99' '3.49' '11.99' '13.99' '34.99' '74.76' '1.49' '32.99' '99.99'
 '14.95' '69.99' '16.99' '79.99' '49.99' '5.0' '44.99' '13.98' '29.96'
 '119.99' '109.99' '149.99' '771.71' '21.99' '89.99' '0.98' '139.92'
 '4.29' '64.99' '54.99' '74.99' '0.89' '0.5' '299.99' '1.29' '3.0' '15.0'
 '5.49' '23.99' '49.0' '20.99' '10.93' '1.3900000000000001' '36.99' '4.49'
 '2.0' '4.0' '9.0' '234.99' '1.9500000000000002' '1.5' '199.0' '189.0'
 '6.66' '27.99' '10.49' '129.99' '179.0' '26.99' '399.99' '31.99' '399.0'
 '20.0' '40.0' '3.33' '199.99' '22.99' '320.0' '38.85' '71.7' '59.95'
 '995.0' '27.49' '3.39' '6.0' '19.95' '499.99' '16.06' '4.68' '131.4'
 '44.98' '202.76' '1.0' '2.3' '0.9500000000000001' '172.24' '249.99'
 '2.9699999999999998' '10.96' '10.0' '30.0' '2.66' '6.48' '19.29' '11.15

In [22]:
missing_values_table(df_games)

Your selected dataframe has 12 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
publisher,8069,25.1
developer,3297,10.3
genres,3281,10.2
release_date,2067,6.4
price,1377,4.3
specs,668,2.1
tags,163,0.5
title,2,0.0
reviews_url,2,0.0
id,2,0.0


In [23]:
#tenemos valores nulos. Lo rellenamos con el publusher ya que en gran medida el publisher y la empresa developer son la misma. De todas maneras solo estamos 
#achicando en un 0.01% la cantidad de valores nulos.

df_games['developer'] = df_games['developer'].fillna(df_games['publisher'])

In [24]:
missing_values_table(df_games)

Your selected dataframe has 12 columns.
There are 10 columns that have missing values.


,Missing Values,% of Total Values
publisher,8069,25.1
genres,3281,10.2
developer,3232,10.1
release_date,2067,6.4
price,1377,4.3
specs,668,2.1
tags,163,0.5
title,2,0.0
reviews_url,2,0.0
id,2,0.0


In [25]:
#relleno con NoData
df_games['developer'] = df_games['developer'].fillna('NoData')
df_games['release_date'] = df_games['release_date'].fillna('NoData')

In [26]:
#elimino columna que no voy a utilizar
df_games.drop(columns= 'publisher', inplace=True)

In [27]:
#vemos que hay varios missing values dentro de la columna price, pecs, tags y genres. 
missing_values_table(df_games)

Your selected dataframe has 11 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
genres,3281,10.2
price,1377,4.3
specs,668,2.1
tags,163,0.5
title,2,0.0
reviews_url,2,0.0
id,2,0.0


Las columnas "tags", "specs", y "genere" son altamente similarres. Una primera aproximación a esto va a ser la de unir las 3 columnas en una sola, de esta forma evito valores faltantes que vemos que hay varios. 

Encontramos algunos missing values (solo un 4.3%) lo cual no nos afectaria tanto eliminar estos missing values del dataframe sin embargo podemos intentar encontrar algunos patrones para determinar si tengo algunos juego que esten carentes de precio por ser gratis.

In [28]:
#encontramos el mismo problema de antes, esta formado por un string, no una lista

print(df_games['tags'][88310])

print(type(df_games['tags'][88310]))

['Strategy', 'Action', 'Indie', 'Casual', 'Simulation']
<class 'str'>


In [29]:
#reemplazamos los nan por un string que no sea valido para python para poder tratarlos

for column in ['tags', 'specs', 'genres']:
    df_games[column] = df_games[column].fillna('___nan___')

In [30]:
#convertimos las columnas de str a listas

for column in ['tags', 'specs', 'genres']:
    df_games[column] = df_games[column].apply(lambda x: ast.literal_eval(x) if x != '___nan___' else [])

In [31]:
#FUNCIONA!

print(df_games['tags'][88310])

print(type(df_games['tags'][88310]))

['Strategy', 'Action', 'Indie', 'Casual', 'Simulation']
<class 'list'>


In [32]:
# creamos una funcion que nos va a tomar los elementos de de specs, genres y tags y los ponde dentro 
# de una sola columna
def concatenate_and_remove_duplicates(lst):
    ''' 
    esta funcion concatena los elementos de las columnas que son formados por lista y devuelve un set
    Los sets no admiten valores repetidos lo cual me permite concatenar genre, tags y specs en una sola columna.
    
    '''
    combined_list = []
    for sublist in lst:
        if isinstance(sublist, list):  # Verificar si el valor es una lista
            combined_list.extend(sublist)
    return list(set(combined_list))

In [33]:

df_games['atributos'] = df_games.apply(lambda row: concatenate_and_remove_duplicates([row['specs'], row['tags'], row['genres']]), axis=1)

In [34]:
df_games.head()

,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,atributos
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,"[Casual, Strategy, Simulation, Indie, Single-p..."
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL,"[PvP, Replay Value, Character Customization, F..."
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,"[Casual, Stats, Simulation, Indie, Single-play..."
88313,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,"[Adventure, Action, Single-player, Casual]"
88314,[],Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NoData,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,NoData,"[Casual, Tracked Motion Controllers, HTC Vive,..."


In [35]:
#ahora creo unnuevo dataframe unicamente con las filas que no tenian precio
nan_in_col  = df_games[df_games['price'].isna()]

In [36]:

imputer = nan_in_col.copy()

In [37]:
#con este imputer voy a intentar imputar precio a algunos valores que no lo tienen
imputer

,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,atributos
88319,"[Casual, Indie, Racing, Simulation]",Race,http://store.steampowered.com/app/768800/Race/,2018-01-04,"[Indie, Casual, Simulation, Racing]",http://steamcommunity.com/app/768800/reviews/?...,"[Single-player, Multi-player, Partial Controll...",NaN,0.0,768800.0,RewindApp,"[Casual, Partial Controller Support, Racing, S..."
88320,"[Action, Indie, Simulation, Early Access]",Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,Soon..,"[Early Access, Action, Indie, Simulation, Surv...",http://steamcommunity.com/app/768570/reviews/?...,[Single-player],NaN,1.0,768570.0,Qucheza,"[Early Access, Simulation, Indie, Survival, Si..."
88341,"[Casual, Indie, Strategy]",Lost Cities,http://store.steampowered.com/app/520680/Lost_...,2018-01-01,"[Casual, Indie, Strategy, Card Game, Board Gam...",http://steamcommunity.com/app/520680/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",NaN,0.0,520680.0,BlueLine Games,"[Casual, Local Multi-Player, Steam Trading Car..."
88342,[Action],Twisted Enhanced Edition,http://store.steampowered.com/app/690410/Twist...,2018-01-01,"[Action, Platformer, Side Scroller]",http://steamcommunity.com/app/690410/reviews/?...,"[Single-player, Full controller support]",NaN,0.0,690410.0,Games by Brundle,"[Full controller support, Platformer, Side Scr..."
88344,"[Action, Casual, Indie, Massively Multiplayer,...",Tactics Forever,http://store.steampowered.com/app/413120/Tacti...,2018-01-01,"[Casual, Action, Massively Multiplayer, Indie,...",http://steamcommunity.com/app/413120/reviews/?...,"[Online Multi-Player, MMO, Cross-Platform Mult...",NaN,0.0,413120.0,ProjectorGames,"[Casual, Partial Controller Support, Massively..."
...,...,...,...,...,...,...,...,...,...,...,...,...
120409,"[Action, Indie]",Infinos Gaiden,http://store.steampowered.com/app/771070/Infin...,2018-01-19,"[Action, Indie, Shoot 'Em Up]",http://steamcommunity.com/app/771070/reviews/?...,"[Single-player, Partial Controller Support]",NaN,0.0,771070.0,Picorinne Soft,"[Partial Controller Support, Indie, Single-pla..."
120421,[Sports],International Online Soccer,http://store.steampowered.com/app/90007/Intern...,2002-01-01,"[Sports, Mod]",http://steamcommunity.com/app/90007/reviews/?b...,"[Multi-player, Mods (require HL2)]",NaN,0.0,90007.0,I.O.S. Team,"[Mods (require HL2), Mod, Multi-player, Sports]"
120433,"[Casual, Indie, Simulation, Sports, Early Access]",Cricket Club,http://store.steampowered.com/app/772180/Crick...,January 2018,"[Early Access, Indie, Casual, Simulation, Sports]",http://steamcommunity.com/app/772180/reviews/?...,[Single-player],NaN,1.0,772180.0,VersoVR,"[Casual, Early Access, Simulation, Indie, Sing..."
120435,"[Action, Adventure, Indie]",The spy who shot me™,http://store.steampowered.com/app/771810/The_s...,2018-10-01,"[Action, Adventure, Indie]",http://steamcommunity.com/app/771810/reviews/?...,"[Single-player, Captions available]",NaN,0.0,771810.0,Retro Army Limited,"[Captions available, Adventure, Indie, Single-..."


In [38]:
#aislo en otro dataframe los juegos gratis
juegos_gratis = df_games[df_games['price'] == '0.00']

In [39]:
#y analizo cuales son los atributos ("combinacion de genre, specs y tags") de los juegos gratis
df = pd.DataFrame(juegos_gratis["atributos"])

In [40]:
df["atributos"]

88311     [PvP, Replay Value, Character Customization, F...
88312     [Casual, Stats, Simulation, Indie, Single-play...
88321     [Casual, HTC Vive, Tracked Motion Controllers,...
88370                [Casual, Puzzle, Single-player, Indie]
88501     [Mod, Multi-player, FPS, Steam Achievements, M...
                                ...                        
120317    [Casual, Partial Controller Support, Indie, Si...
120387    [Casual, Seated, Early Access, Tracked Motion ...
120410    [Massively Multiplayer, Strategy, RPG, In-App ...
120411    [Partial Controller Support, Anime, Gore, Simu...
120431                    [Tutorial, Design & Illustration]
Name: atributos, Length: 1908, dtype: object

In [41]:
from collections import Counter
import ast
#df['atributos'] = df['atributos'].apply(ast.literal_eval)
atributos_counter = Counter()

In [42]:
for atributo_lista in df['atributos']:
    atributos_counter.update(atributo_lista)

In [43]:
atributos_mas_comunes = atributos_counter.most_common()

asi aislo los atributos mas comunes enlos juegos gratis. En este momento aplico algo de experiencia propia en el área.
uso este iterador para contar que atributos se repiten mas y determinar cuales son las palabras claves que mas aparecen en los juegos gratuitos de forma manual y segun mi criterio


In [ ]:
lista= []
for atributo, count in atributos_mas_comunes:
    lista.append(atributo)
    print(f"{atributo}: {count}")

In [45]:
palabras_clave_gratuitas = ['Free to Play', 'Multi-player', 'Downloadable Content', 'Steam Trading Cards',
                            'Multiplayer', 'Online Multi-Player', 'Massively Multiplayer', 'Co-op', 'MMO',
                            'Cross-Platform Multiplayer', 'Online Co-op', 'MMO', 'MMORPG', 'Visual Novel',
                            'Card Game', 'Memes', 'Mod', 'MOBA', 'Trading Card Game', 'Competitive', "Game demo", "Utilities"]

In [46]:
#aplico con una funcion

def es_juego_gratuito(atributos):
    for palabra_clave in palabras_clave_gratuitas:
        if palabra_clave in atributos:
            return True
    return False

In [47]:
#ahora aplico a mi imputador la funcion creada para determinar si algo es gratuito o no
imputer['es_gratuito'] = imputer['atributos'].apply(es_juego_gratuito)

In [48]:
#relleno valor price por 0.00
imputer.loc[imputer['es_gratuito'], 'price'] = '0.00'

In [49]:
#imputo valores al df original a partir de mi imputador
df_games['price'] = df_games['price'].fillna(imputer['price'])

De esta manera baje a 1% la cantidad de valores faltantes de la columna price y con un motivo logico detras.


In [52]:

missing_values_table(df_games)

Your selected dataframe has 12 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
price,306,1.0
title,2,0.0
reviews_url,2,0.0
id,2,0.0


In [53]:
df_games = df_games.dropna()

In [54]:
missing_values_table(df_games)

Your selected dataframe has 12 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


Ahora voy a normalizar la columna release_date, utilizo un iterador para imprimir en pantalla los todos los valores unicos, luego hafo una seleccion manual para reemplazarlos.


In [ ]:

list = pd.unique(df_games['release_date'])

lista = sorted(list, reverse=True)

for i in lista:
    print(i)

In [56]:
#Estos nombres hacen referencia a que no tienen fecha de salida aún

df_games.replace(["预热群52756441",
                    "soon",
                    "early access",
                    "coming soon",
                    "When it's done!",
                    "When it is ready",
                    "When it is finished",
                    "To be Announced",
                    "To Be Announced",
                    "TBD",
                    "TBA",
                    "Soon™",
                    "Soon",
                    "SOON™",
                    "SOON",
                    "Release Date TBA",
                    "Not yet available",
                    "Early Access soon",
                    "Early Access Starting Soon!",
                    "Demo coming soon.",
                    "Coming soon..",
                    "Coming soon",
                    "Coming Soon/Próximamente",
                    "Coming Soon",
                    "Beta测试已开启", #Las pruebas beta están en marcha
                    "Play Beta in demo!"
                    ], 'NoData', inplace=True)

In [57]:
#promedio a mitad de año

df_games.replace(['Winter 2017',
                "When it's done (2017)",
                "TBA 2017",
                "Summer 2017",
                "Q4 2017"
                "Hitting Early Access in 2017",
                "Fall 2017",
                "End 2017",
                "Early 2017",
                "December 2017",
                "Dec 2017",
                "Apr 2017",
                "Hitting Early Access in 2017"], '2017-07-07', inplace=True)

In [58]:
df_games.replace(["Spring 2018",
                  "Q1 2018",
                  "Mar 2018",
                  "First quarter of 2018",
                  "August 2018",
                  "2018年初頭発売予定",
                "2018年1月",
                "2018",
                "15.01.2018"], '2018-07-01', inplace=True)

In [59]:
df_games.replace(["Sep 2014",
                  "Nov 2014",
                  "May 2014",
                  "Jul 2014",
                  "Aug 2014",], ["2014-09-01", "2014-11-01", "2014-05-01", "2014-07-01", "2014-08-01"],  inplace=True)

df_games.replace(["Sep 2009",
                  "Oct 2009",
                  "Jun 2009"], ["2009-09-01", "2009-10-01", "2009-06-01"],  inplace=True)

df_games.replace(["Oct 2016",
                  "Nov 2016",
                  "Jun 2016",
                  "Jul 2016",
                  "Apr 2016"], ["2016-10-01", "2016-11-01", "2016-06-01", "2016-07-01", "2016-04-01"],  inplace=True)

df_games.replace(["Apr 2015",
                  "Aug 2015",
                  "Dec 2012",
                  "Early 2018",
                  "Q4 2017",
                  "Oct 2010",
                  "Nov 2017",
                    "May 2015",
                    "Mar 2010",
                    "Jun 2015",
                    "Jul 2017",
                    "Jul 2010",
                    "January 2019",
                    "Jan 2017",
                    "Jan 2015",
                    "Jan 2010",
                    "Feb 2015",
                    "Feb 2013",
                    "Feb 2011"], ["2015-04-01", "2015-08-01", "2012-12-01", "2018-02-01","2017-10-01","2010-10-01","2017-10-01","2015-05-01",
                                  "2010-03-01", "2015-06-01", "2017-07-01","2010-07-01", "2019-01-01", "2017-01-01", "2015-01-01", "2010-01-01","2015-02-01","2013-02-01","2011-02-01"],  inplace=True)

In [60]:
df_games.replace(["2017 Q4",
                    "2017 Q1",
                    "2017",
                    "2016",
                    "1st Quarter 2018"], ["2017-10-01", "2017-02-01", "2017-01-01","2016-01-01","2018-03-01"],  inplace=True)

In [ ]:
list = pd.unique(df_games['release_date'])

lista = sorted(list, reverse=True)

for i in lista:
    print(i)

In [72]:
#normalizo el nombre para que sea el mismo en todas las tablas
df_games.rename(columns={'id' : "item_id"}, inplace=True)

In [62]:
#exporto mi dataset totalmente limpio y normalizado y asi poder usarlo para trabajar 

df_games.to_csv('df_games_clean.csv', index=False)

### user_reviews

In [103]:
import pandas as pd
import nltk
import numpy as np
from datetime import datetime


nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')


def missing_values_table(df):
    mis_val = df.isna().sum()
    mis_val_percent = 100 * df.isna().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns


df_reviews = pd.read_csv("data\df_reviews.csv")


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gasto\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\gasto\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gasto\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gasto\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [104]:
df_reviews

,Unnamed: 0,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability. In my opi...
1,1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,It's unique and worth a playthrough.
2,2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,3,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,4,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...,...
59328,59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,Posted July 10.,NaN,70.0,No ratings yet,True,a must have classic from steam definitely wort...
59329,59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,Posted July 8.,NaN,362890.0,No ratings yet,True,this game is a perfect remake of the original ...
59330,59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,NaN,273110.0,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,Posted July 20.,NaN,730.0,No ratings yet,True,:D


In [105]:
missing_values_table(df_reviews)

Your selected dataframe has 10 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values
last_edited,53193,89.7
funny,51182,86.3
review,58,0.1
posted,28,0.0
item_id,28,0.0
helpful,28,0.0
recommend,28,0.0


In [106]:
#rellenamos los valores nulos en la columna 'review' con el string "neutral" para poder tratarlo.

df_reviews['review'] = df_reviews['review'].fillna('neutral')

In [107]:
#testeamos
ejemplo = df_reviews['review'][8526]
print(ejemplo)

99% of n user reviews for this game are positive.It was the first thought when I bought this game, and I have no regrets.Longer Version (Playtime of the game itself):I would still recommend this game even how it's not all filled with features and such. Probably when it gets released from early access, it will be much more fun to play.SPOILERS AHEAD!At first, it starts out bland, but then you start to automate for ores and turn it into plates, where plates are used to create tools. After manually researching, you decide you should automate Science. (At this point let's say we have electricity) After auto research, a biter attack happens. You only have a gun to protect yourself as a gun turret is still being built. You then defend your base, and automate more science. 10-30 hours later, you are already building a rocket silo, where you finish the game. You then decide to install mods, where you go all over again with a new experience. It never ends.In a nutshell, words cannot express how

#### MODELO VADER PARA ANALIZAR SENTIMIENTOS


Este modelo de analisis de sentimientos es muy efectivo para utilizar cuando no tenemos etiquetas en las que basarnos, nos va a devolver una tabla por cada valor texto analizado diciendonos en que medida el texto literal es interpretado como negativo, neutro, o positivo. Tambien nos devuelve un indice que va de -1 para negativo a 1 para positivo, donde el 0 marca la neutralidad absoluta.

In [108]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [109]:
# testeando algunas oraciones
print(sia.polarity_scores("My name is Gaston and i am tryng to make a sentiment analyzer with python. it is fun!"))

print(sia.polarity_scores("I am so happy!"))

print(sia.polarity_scores("Ordered a waffle to find a fingernail and a hair on my food, absolutely shocking customer sevice as i was told O would have to wait another hour to get a fresh one. I asked for a refund but never received one wouldn't send my worst enemy here"))

print(sia.polarity_scores("great game, Steam please give it to me but anyways so fun with firends or without it and steam plz gift it to me i really want it because its such a good game overall 100/10 i rate it because its so amazing"))

print(sia.polarity_scores("neutral"))

{'neg': 0.0, 'neu': 0.807, 'pos': 0.193, 'compound': 0.5562}
{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}
{'neg': 0.138, 'neu': 0.737, 'pos': 0.125, 'compound': -0.1669}
{'neg': 0.0, 'neu': 0.534, 'pos': 0.466, 'compound': 0.9816}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [110]:
#aplicamos el modelos
res = {}

for i, row in tqdm(df_reviews.iterrows(), total=len(df_reviews)):
    text = row['review']
    myid = row['Unnamed: 0']
    res[myid] = sia.polarity_scores(text)


  0%|          | 0/59333 [00:00<?, ?it/s]

In [111]:
sent = pd.DataFrame(res).T

sent

,neg,neu,pos,compound
0,0.077,0.681,0.242,0.8481
1,0.000,0.678,0.322,0.2263
2,0.000,0.704,0.296,0.9117
3,0.066,0.721,0.213,0.9566
4,0.041,0.768,0.191,0.9708
...,...,...,...,...
59328,0.000,0.566,0.434,0.5574
59329,0.026,0.692,0.282,0.9786
59330,0.141,0.502,0.357,0.7827
59331,0.000,0.000,1.000,0.5106


In [112]:
df_reviews = df_reviews.merge(sent['compound'], left_index=True, right_index=True)

In [113]:
#mergeamos las dos tablas, el coumpound es el valos que nos interesa de los 4 que nos da el modelo
df_reviews.head()

,Unnamed: 0,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,compound
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability. In my opi...,0.8481
1,1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,It's unique and worth a playthrough.,0.2263
2,2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0.9117
3,3,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,0.9566
4,4,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,0.9708


In [116]:
#definimos como va a ser los parametros, en este nuestro caso 0.15 es el punto que nos va a decir si es positivo, neutral o negativo

def map_sentiment(value):
    if value < -0.15:
        return 0  # Negativo
    elif -0.15 <= value <= 0.15:
        return 1  # Neutral
    else:
        return 2  # Positivo

In [117]:
df_reviews['sentimiento'] = df_reviews['compound'].apply(map_sentiment)

In [118]:
df_reviews.drop(columns=['compound','review','Unnamed: 0','last_edited','funny',], inplace = True)

In [119]:
df_reviews.drop(columns=['helpful'], inplace = True)

In [120]:
#chequemaos por nulos, vemos que son muy pocos y no podemos de ninguna forma predecir si los valores faltantes van a ser positivos, negativos o neutros asi que los eliminamos

missing_values_table(df_reviews)

Your selected dataframe has 6 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
posted,28,0.0
item_id,28,0.0
recommend,28,0.0


In [121]:
nan_in_col  = df_reviews[df_reviews['recommend'].isna()]

In [122]:
nan_in_col

,user_id,user_url,posted,item_id,recommend,sentimiento
137,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,1
177,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,1
2559,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,1
10080,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,1
13767,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,1
15493,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,1
19184,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,1
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,1
25056,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,1
26257,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,1


Como son 3 columnas vitales para poder analizarlas y los valores faltantes son en esas 3 columnas, decido elimirlas. Ademas si vemos a que porcentaje pertenecen los nulos sobre el total vemos que es practicamente nada.

In [123]:
df_reviews = df_reviews.dropna()

In [124]:
#dataframe limpio sin valores nulos

missing_values_table(df_reviews)

Your selected dataframe has 6 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [125]:
df_reviews

,user_id,user_url,posted,item_id,recommend,sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250.0,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200.0,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110.0,True,2
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610.0,True,2
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300.0,True,2
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 10.,70.0,True,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,Posted July 8.,362890.0,True,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 3.,273110.0,True,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,Posted July 20.,730.0,True,2


In [ ]:
#comenzamamos a normalizar la  columna "posted", que hace referencia a la fecha que pose posteo el comentario

list = pd.unique(df_reviews['posted'])

lista = sorted(list, reverse=True)

for i in lista:
    print(i)

In [128]:
#creamos funcion para cambiar el tipo de dato a fecha

def convert_to_date(date_str):
    if isinstance(date_str, str):
        try:
            return datetime.strptime(date_str, "Posted %B %d, %Y.")
        except ValueError:
            return np.nan
    else:
        return np.nan

In [129]:
df_reviews["posted"] = df_reviews["posted"].apply(convert_to_date)


In [ ]:
list = pd.unique(df_reviews['posted'])

lista = sorted(list, reverse=True)

for i in lista:
    print(i)

In [131]:
df_reviews['posted'] =  df_reviews["posted"].dt.date

In [132]:
df_reviews

,user_id,user_url,posted,item_id,recommend,sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250.0,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200.0,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110.0,True,2
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610.0,True,2
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300.0,True,2
...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,70.0,True,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,362890.0,True,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,273110.0,True,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,730.0,True,2


In [133]:
#dataframe limpio y listo para usarse

df_reviews.to_csv('df_reviews_clean.csv', index=False)

### user_items

In [134]:
import pandas as pd

def missing_values_table(df):
    mis_val = df.isna().sum()
    mis_val_percent = 100 * df.isna().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

df_items = pd.read_csv('data\df_items.csv')

In [135]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Unnamed: 0        int64  
 1   user_id           object 
 2   items_count       int64  
 3   steam_id          int64  
 4   user_url          object 
 5   item_id           float64
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(3), int64(3), object(3)
memory usage: 355.0+ MB


In [136]:
#vemos que coincide el item_counts con la cantidad de registros

df_items.head(280)

,Unnamed: 0,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0
1,1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20.0,Team Fortress Classic,0.0,0.0
2,2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.0,Day of Defeat,7.0,0.0
3,3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40.0,Deathmatch Classic,0.0,0.0
4,4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50.0,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
275,275,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,379720.0,DOOM,793.0,0.0
276,276,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,273350.0,Evolve Stage 2,58.0,0.0
277,277,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,10.0,Counter-Strike,0.0,0.0
278,278,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,80.0,Counter-Strike: Condition Zero,0.0,0.0


son menos del 0.3% de los datos los faltantes, son extremadamente pocos y lo mas probable es que no nos interfiera en nuestro analisis


In [137]:
missing_values_table(df_items)

Your selected dataframe has 9 columns.
There are 4 columns that have missing values.


,Missing Values,% of Total Values
item_id,16806,0.3
item_name,16806,0.3
playtime_forever,16806,0.3
playtime_2weeks,16806,0.3


In [138]:
nan_in_col  = df_items[df_items['item_id'].isna()]

In [139]:
#como vemos, no tenemos referencia de item alguna.
#PD: contador de "Unnamed: 0" = 3

nan_in_col

,Unnamed: 0,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
3733,3733,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN,NaN
3849,3849,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
6019,6019,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN,NaN
6523,6523,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN,NaN
7237,7237,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5169470,5169470,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169471,5169471,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169472,5169472,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN,NaN
5170006,5170006,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN,NaN


In [140]:
df_items = df_items.dropna()

In [141]:
df_items

,Unnamed: 0,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0
1,1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20.0,Team Fortress Classic,0.0,0.0
2,2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.0,Day of Defeat,7.0,0.0
3,3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40.0,Deathmatch Classic,0.0,0.0
4,4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50.0,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5170009,5170009,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,346330.0,BrainBread 2,0.0,0.0
5170010,5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330.0,All Is Dust,0.0,0.0
5170011,5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490.0,One Way To Die: Steam Edition,3.0,3.0
5170012,5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570.0,You Have 10 Seconds 2,4.0,4.0


In [142]:
#dataframe limpio y listo.
missing_values_table(df_items)

Your selected dataframe has 9 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [143]:
df_items.to_csv('df_items_clean.csv', index=False)